In [ ]:
!pip install timm torch torchvision opencv-python

In [ ]:
!pip install nvidia-smi

In [ ]:
!unzip -d . DeepTomography.zip
%cd DeepTomography

In [5]:
!ls

BASELINE_config.yaml  model	 test.py      utils
data		      optim	 train.py     VANILLA_config.yaml
default_config.yaml   scheduler  train_rc.py


In [6]:
import torch
from model import create_model
from optim import create_optimizer
from utils import count_parameters
from tqdm.notebook import tqdm
import nvidia_smi

In [7]:
class Config(object):
  def __init__(self, **kwargs):
    self.__dict__ = kwargs
    for key, value in kwargs.items():
      setattr(self, key, value)

In [8]:
output_size = 256

args = Config(**dict(
      batch_size = 8,
      model_name = 'unet',
      backbone_name = 'vanilla',
      interpolation = 'bilinear',
      upscaling_layer = 'transposeconv',
      interpolate_sinogram = False,
      broadcast_sinogram = True,
      num_proj = 100,
      encoder_channels = [100, 128, 256, 512, 1024],
      decoder_channels = [512, 256, 128, 64, 64],
      activation = 'ReLU',
      skip_connection = True,
      no_pretrained_backbone = False,
      efficient_block = False,
      final_activation = 'Identity',
      optimizer='adam',
      optimizer_eps=1e-5,
      weight_decay=4e-5,
      momentum=0.9,
      amsgrad=False,
      init_lr=1e-3,
      clip_grad=10.0
      )
)

model = create_model(args, output_size=(output_size,output_size))
print("Model #parameters : {}".format(count_parameters(model)))
model = model.train()
loss_fn = torch.nn.MSELoss(reduction='mean')

DEVICE = 'cpu'
if torch.cuda.is_available():
  model = model.cuda()
  loss_fn = loss_fn.cuda()
  DEVICE = 'cuda'

  nvidia_smi.nvmlInit()
  handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

optimizer = create_optimizer(model.parameters(), args)

input = torch.randn(args.batch_size, args.num_proj, output_size, output_size)
target = torch.randn(args.batch_size, 1, output_size, output_size)

epoch_pbar = tqdm(range(50), total=50)
batch_pbar = tqdm(range(100), total=100, leave=True)

for epoch in epoch_pbar:
  for batch_idx in batch_pbar:
    input = input.to(DEVICE, non_blocking=True)
    target = target.to(DEVICE, non_blocking=True)

    output = model(input)
    loss = loss_fn(output, target)

    optimizer.zero_grad(set_to_none=False)
    loss.backward()
    if args.clip_grad is not None:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)
    optimizer.step()

    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    batch_pbar.set_description(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

[100, 128, 256, 512, 1024]
[512, 256, 128, 64, 64]
Model #parameters : 33417293


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
optimizer.zero_grad(set_to_none=True)
model, input, target, output = None, None, None, None
import gc
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi